In [46]:
import pandas as pd
import numpy as np
import nltk
import json
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
# Import CountVectorizer and TFIDFVectorizer from feature_extraction.text.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [6]:
wine_data=pd.read_csv('test_output_wine.csv')

In [7]:
wine_data.head()

,Unnamed: 0,kind,data
0,0,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
1,1,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
2,2,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
3,3,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
4,4,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."


In [41]:
wine_data.drop(columns = ['Unnamed: 0'], inplace = True)
wine_data.head()

,kind,data
0,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
1,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
2,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
3,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."
4,t3,"{'approved_at_utc': None, 'subreddit': 'wine',..."


In [42]:
2386*2

4772

In [47]:
foo = wine_data.loc[0, 'data']

In [55]:
foo.strip("'")

'{\'approved_at_utc\': None, \'subreddit\': \'wine\', \'selftext\': \'People are going to go out of business if this happens. Lives will be ruined.\\n\\n**Click Here to Email Your U.S. House Representative:**\\n\\nhttps://account.votility.com/enterprise/NAWR/ec/698\\n\\n**Click Here to Email Your U.S. Senators:**\\n\\nhttps://account.votility.com/enterprise/NAWR/ec/697\\n\\n\\n**WRITE THE U.S. TRADE REPRESENTATIVE**\\n \\nBelow is wording for a letter that you can use to write to the U.S. Trade Representative at the website they have created to take input on proposed tariffs.\\n\\n1. Copy the letter below (or add personalization as you like).\\n\\n2. Then click on the link at the end of the letter.\\n\\n3. Then paste the wording into the box provided for writing commentary. Remember to put your name and address in the area reserved for them.\\n\\n\\nThe Honorable Robert Lighthizer\\nU.S. Trade Representative\\nExecutive Office of the President\\n600 17th Street, NW\\nWashington, DC 200

In [15]:
wine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 3 columns):
Unnamed: 0    2386 non-null int64
kind          2386 non-null object
data          2386 non-null object
dtypes: int64(1), object(2)
memory usage: 56.0+ KB


In [33]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = {'english'},
                             max_features = 5000) 

In [18]:
wine_data.drop(wine_data['Unnamed: 0'],axis=1)

KeyError: '[0 1 2 ... 2383 2384 2385] not found in axis'

In [36]:
vectorizer.fit(wine_data['data'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=5000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words={'english'},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [37]:
print('Vocabulary: ')
print(vectorizer.vocabulary_)

Vocabulary: 
{'approved_at_utc': 654, 'none': 3189, 'subreddit': 4216, 'wine': 4850, 'selftext': 3961, 'people': 3399, 'are': 661, 'going': 2330, 'to': 4473, 'go': 2325, 'out': 3322, 'of': 3257, 'business': 929, 'if': 2559, 'this': 4437, 'happens': 2424, 'lives': 2854, 'will': 4844, 'be': 781, 'ruined': 3863, 'click': 1151, 'here': 2464, 'email': 1800, 'your': 4968, 'house': 2512, 'representative': 3776, 'nhttps': 3164, 'account': 536, 'votility': 4715, 'com': 1189, 'enterprise': 1860, 'nawr': 3122, 'ec': 1618, '698': 405, '697': 404, 'write': 4902, 'the': 4418, 'trade': 4509, 'is': 2633, 'wording': 4885, 'for': 2210, 'letter': 2816, 'that': 4415, 'you': 4965, 'can': 983, 'use': 4613, 'at': 684, 'website': 4775, 'they': 4430, 'have': 2433, 'created': 1291, 'take': 4352, 'input': 2603, 'on': 3281, 'proposed': 3586, 'tariffs': 4366, 'n1': 3092, 'copy': 1252, 'below': 804, 'or': 3302, 'add': 545, 'as': 675, 'like': 2827, 'n2': 3093, 'then': 4424, 'link': 2835, 'end': 1837, 'n3': 3097, 'pa

In [38]:
vector = vectorizer.transform(wine_data['data'])

In [39]:
#experimenting with insights found at https://towardsdatascience.com/natural-language-processing-count-vectorization-with-scikit-learn-e7804269bb5e

print('Full vector: ')
print(vector.toarray())

Full vector: 
[[0 3 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


0       {'approved_at_utc': None, 'subreddit': 'wine',...
1       {'approved_at_utc': None, 'subreddit': 'wine',...
2       {'approved_at_utc': None, 'subreddit': 'wine',...
3       {'approved_at_utc': None, 'subreddit': 'wine',...
4       {'approved_at_utc': None, 'subreddit': 'wine',...
                              ...                        
2381    {'approved_at_utc': None, 'subreddit': 'wine',...
2382    {'approved_at_utc': None, 'subreddit': 'wine',...
2383    {'approved_at_utc': None, 'subreddit': 'wine',...
2384    {'approved_at_utc': None, 'subreddit': 'wine',...
2385    {'approved_at_utc': None, 'subreddit': 'wine',...
Name: data, Length: 2386, dtype: object